In [82]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd

In [2]:
data = pd.read_csv('FakeNewsNet.csv')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          23196 non-null  object
 1   news_url       22866 non-null  object
 2   source_domain  22866 non-null  object
 3   tweet_num      23196 non-null  int64 
 4   real           23196 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 906.2+ KB


In [4]:
print(data.shape)
data.head()

(23196, 5)


,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [11]:
data.describe()

,tweet_num,real
count,23196.000000,23196.000000
mean,88.956803,0.751897
std,488.694592,0.431921
min,0.000000,0.000000
25%,11.000000,1.000000
50%,37.000000,1.000000
75%,65.000000,1.000000
max,29060.000000,1.000000


In [5]:
data.isnull().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [8]:
data['real'].value_counts()

#1--> Real 
#0--> Fake

1    17441
0     5755
Name: real, dtype: int64

# Preprocessing URL

In [66]:
data['news_url'].astype(str)
data['secure']=data['news_url'].apply(lambda x: str(x).split(':')[0])

data['check_secure'] = data['secure'].apply(lambda x: 1 if str(x) in ('http','https') else 0)

In [68]:
data.drop(['title','news_url','source_domain','secure'], axis=1,inplace=True)

In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   tweet_num     23196 non-null  int64
 1   real          23196 non-null  int64
 2   check_secure  23196 non-null  int64
dtypes: int64(3)
memory usage: 543.8 KB


# Splitting Dataset and Scaling



In [71]:
X = data.drop(columns='real', axis=1)
Y = data['real']

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [73]:
X_train.shape

(18556, 2)

In [74]:
# Scaling values

sc = StandardScaler()
X_train[['tweet_num']] = sc.fit_transform(X_train[['tweet_num']])
X_test[['tweet_num']] = sc.transform(X_test[['tweet_num']])

C:\Users\Mohamed Ashraf\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\Mohamed Ashraf\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


# Training the K-NN model on the Training set

In [75]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, Y_train)

KNeighborsClassifier()

# Prediction and Score

In [89]:
pred = classifier.predict(X_test)

print(accuracy_score(Y_train, classifier.predict(X_train)))
accuracy_score(Y_test, pred)

0.982377667600776


0.9816810344827587

In [83]:
print(confusion_matrix(Y_test, pred))

[[1113   61]
 [  24 3442]]


In [85]:
print(classification_report(Y_test, pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1174
           1       0.98      0.99      0.99      3466

    accuracy                           0.98      4640
   macro avg       0.98      0.97      0.98      4640
weighted avg       0.98      0.98      0.98      4640

